In [ ]:
df = pd.DataFrame(index=ticker_data_1.index)
df[ticker1] = ticker_data_1["Close"]
df[ticker2] = ticker_data_2["Close"]

model = sm.OLS(df[ticker1], df[ticker2]) 
res = model.fit()
beta_hr = res.params[0]

# Calculate the residuals of the linear combination
df["res"] = df[ticker1] - beta_hr * df[ticker2]

# Calculate and output the CADF test on the residuals
cadf = ts.adfuller(df["res"])

zscore = ((df["res"] - df["res"].mean())/ df["res"].std())[-1]
df["z-s"] = ((df["res"] - df["res"].mean())/ df["res"].std())

In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import numpy as np

x = rw.get_window(group[0].symbol)
y = rw.get_window(group[1].symbol)

last_2_prices_x = x[-2:]
last_2_prices_y = y[-2:]

time_indices_x = np.arange(len(x)-2, len(x))
time_indices_y = np.arange(len(y)-2, len(y))

model_x = LinearRegression()
model_y = LinearRegression()
model_x.fit(time_indices_x.reshape(-1, 1), last_2_prices_x)
model_y.fit(time_indices_y.reshape(-1, 1), last_2_prices_y)

slope_x = model_x.coef_[0]
slope_y = model_y.coef_[0]

signs_opposite = np.all(slope_x == -slope_y)


In [57]:
from sklearn.linear_model import LinearRegression
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
import statsmodels.api as sm
import yfinance as yf
import pandas as pd
import numpy as np

def normalize_start(df, start):
    # Subtract the 'open' value at 'start' index from all 'open' values
    df['Close'] = df['Close'] - df['Close'].iloc[start]
    return df

def normalize_minmax(df):
    # start does nothing here, just more work to rearrange code everytime i switch
    df['Close'] = (df['Close'] - df['Close'].min()) / (df['Close'].max() - df['Close'].min())
    return df

def read_and_normalize(filename, start):
    # Read CSV
    df = pd.read_csv(filename, parse_dates=['Date'])
    df_copy = df.copy()
    df1_copy = df.copy()
    df2_copy = df.copy()
    # df = pd.read_csv(filename, parse_dates=['timestamp'])
    # Normalize
    df_n_start = normalize_start(df1_copy, start)
    df_n_minmax = normalize_minmax(df2_copy)
    return df_n_start, df_n_minmax, df_copy

def filter_by_index(df_normalized, df_n_minmax, df, start, end):
    return df_normalized.iloc[start:end], df_n_minmax.iloc[start:end], df.iloc[start:end]

def check_opposite_signs_fixed(x, y, idx):
    # Using the last two days to check the signs.
    last_2_prices_x = x.iloc[idx-2:idx]
    last_2_prices_y = y.iloc[idx-2:idx]

    time_indices_x = pd.Series(range(2))  # two points, so just [0, 1]
    time_indices_y = pd.Series(range(2))

    model_x = LinearRegression()
    model_y = LinearRegression()
    model_x.fit(time_indices_x.values.reshape(-1, 1), last_2_prices_x.values)
    model_y.fit(time_indices_y.values.reshape(-1, 1), last_2_prices_y.values)

    slope_x = model_x.coef_[0]
    slope_y = model_y.coef_[0]

    return slope_x == -slope_y

def divide_stocks(stock_data_1, stock_data_2):
    return stock_data_1 / stock_data_2

def OlsBetahrRes(stock_data_1, stock_data_2):
    model = sm.OLS(stock_data_1, stock_data_2) 
    res = model.fit()
    beta_hr = res.params[0]
    residuals = stock_data_1 - beta_hr * stock_data_2
    return residuals

def lin_reg_xy(padas_date_price_x, padas_date_price_y):
    lin_reg_x = []
    lin_reg_y = []
    if len(padas_date_price_x) == len(padas_date_price_y):
        for itr in range(2, len(padas_date_price_x)):
            last_2_prices_x = padas_date_price_x[itr-2:itr]
            last_2_prices_y = padas_date_price_y[itr-2:itr]

            print("type(last_2_prices_x)", type(last_2_prices_x))

            time_indices_x = np.arange(len(padas_date_price_x)-2, len(padas_date_price_x))
            time_indices_y = np.arange(len(padas_date_price_y)-2, len(padas_date_price_y))

            print("type(time_indices_x)", type(time_indices_x))

            model_x = LinearRegression()
            model_y = LinearRegression()
            model_x.fit(time_indices_x.reshape(-1, 1), last_2_prices_x)
            model_y.fit(time_indices_y.reshape(-1, 1), last_2_prices_y)

            slope_x = model_x.coef_[0]
            slope_y = model_y.coef_[0]

            print("type(slope_x)", type(slope_x))
            print("slope_x", slope_x)
            
            lin_reg_x.append(slope_x)
            lin_reg_y.append(slope_y)
    return lin_reg_x, lin_reg_y

def find_combos_ab_res_zero(res_data, zd, x_or_y, above_below:bool):
        gxy = np.full(len(res_data), np.nan)
        index = 0
        if above_below == True:
            for i in zd.index:
                if zd[i] > 0:
                    gxy[index] = i
                index += 1
        else:
            for i in res_data.index:
                if res_data[i] < 0:
                    gxy[index] = i
                index += 1
        wombo_combo = pd.Series(x_or_y, gxy)
        return wombo_combo

def lin_reg_above_below(lst, aboveT_belowF:bool):
        gxy = np.full(len(lst), np.nan)
        index = 0
        if aboveT_belowF == True:
            for i in lst:
                if i > 0:
                    gxy[index] = i
                index += 1
        else:
            for i in lst:
                if i < 0:
                    gxy[index] = i
                index += 1
        return gxy

def compute_sma(data, window_size=50):
    return data.rolling(window=window_size).mean()

def compute_ema(data, span=20):
    return data.ewm(span=span, adjust=False).mean()

def entire_zscore(x_val,y_val):
    residuals_func = OlsBetahrRes(x_val, y_val)
    cadf = ts.adfuller(residuals_func)
    print(cadf)
    zscore = ((residuals_func - residuals_func.mean())/ residuals_func.std())
    return zscore, cadf


def plot(df1_start, df1_n_minmax, df1_copy, df2_start, df2_n_minmax, df2_copy, start):

    fig, axs = plt.subplots(12, figsize=(18,30))

    zscore, cadf = entire_zscore(df1_copy['Close'], df2_copy['Close'])
    zscore_s, cadf_s = entire_zscore(df1_start['Close'], df2_start['Close'])
    zscore_m, cadf_m = entire_zscore(df1_n_minmax['Close'], df2_n_minmax['Close'])

    # dividing the two stocks
    div_stk = df1_copy['Close'] / df2_copy['Close']
    div_stk_s = df1_start['Close'] / df2_start['Close']
    div_stk_m = df1_n_minmax['Close'] / df2_n_minmax['Close']

    # res via division
    ratio_data = divide_stocks(df1_copy['Close'], df2_copy['Close'])
    ratio_data_s = divide_stocks(df1_start['Close'], df2_start['Close'])
    ratio_data_m = divide_stocks(df1_n_minmax['Close'], df2_n_minmax['Close'])

    # residuals via ols -> betahr ->  res
    res_data = OlsBetahrRes(df1_copy['Close'], df2_copy['Close'])
    res_data_s = OlsBetahrRes(df1_start['Close'], df2_start['Close'])
    res_data_m = OlsBetahrRes(df1_n_minmax['Close'], df2_n_minmax['Close'])

    # ema of residuals
    res_ema = compute_ema(res_data)
    res_ema_s = compute_ema(res_data_s)
    res_ema_m = compute_ema(res_data_m)

    combo_dates_x = find_combos_ab_res_zero(
                                            res_data=res_data,
                                            zd=res_data, 
                                            x_or_y=df1_copy['Close'],
                                            above_below=False
    )
    combo_dates_x_s = find_combos_ab_res_zero(
                                            res_data=res_data_s,
                                            zd=res_data_s, 
                                            x_or_y=df1_copy['Close'],
                                            above_below=False
    )
    combo_dates_x_m = find_combos_ab_res_zero(
                                            res_data=res_data_m,
                                            zd=res_data_m, 
                                            x_or_y=df1_copy['Close'],
                                            above_below=False
    )


    combo_dates_y = find_combos_ab_res_zero(
                                            res_data=res_data,
                                            zd=res_data, 
                                            x_or_y=df2_copy['Close'],
                                            above_below=True
    )
    combo_dates_y_s = find_combos_ab_res_zero(
                                            res_data=res_data_s,
                                            zd=res_data_s, 
                                            x_or_y=df2_copy['Close'],
                                            above_below=True
    )
    combo_dates_y_m = find_combos_ab_res_zero(
                                            res_data=res_data_m,
                                            zd=res_data_m, 
                                            x_or_y=df2_copy['Close'],
                                            above_below=True
    )


    lin_reg_x, lin_reg_y = lin_reg_xy(df1_copy["Close"], df2_copy["Close"])
    lin_reg_x_s, lin_reg_y_s = lin_reg_xy(df1_start["Close"], df2_start["Close"])
    lin_reg_x_m, lin_reg_y_m = lin_reg_xy(df2_n_minmax["Close"], df2_n_minmax["Close"])

    buy_x_below_lin_reg = lin_reg_above_below(lin_reg_x,False)
    buy_y_above_lin_reg = lin_reg_above_below(lin_reg_y,True)
    buy_x_below_lin_reg_s = lin_reg_above_below(lin_reg_x_s,False)
    buy_y_above_lin_reg_s = lin_reg_above_below(lin_reg_x_s,True)
    buy_x_below_lin_reg_m = lin_reg_above_below(lin_reg_x_m,False)
    buy_y_above_lin_reg_m = lin_reg_above_below(lin_reg_x_m,True)

    pd_date_lin_reg_x = pd.Series(lin_reg_x, df1_copy.index[2:])
    pd_date_lin_reg_y = pd.Series(lin_reg_y, df2_copy.index[2:])
    pd_date_lin_reg_x = pd.Series(lin_reg_x_s, df1_copy.index[2:])
    pd_date_lin_reg_y = pd.Series(lin_reg_y_s, df2_copy.index[2:])
    pd_date_lin_reg_x = pd.Series(lin_reg_x_m, df1_copy.index[2:])
    pd_date_lin_reg_y = pd.Series(lin_reg_y_m, df2_copy.index[2:])

    pd_buy_x_below_lin_reg = pd.Series(buy_x_below_lin_reg, df1_copy.index[2:])
    pd_buy_y_above_lin_reg = pd.Series(buy_y_above_lin_reg, df2_copy.index[2:])
    pd_buy_x_below_lin_reg_s = pd.Series(buy_x_below_lin_reg_s, df1_copy.index[2:])
    pd_buy_y_above_lin_reg_s = pd.Series(buy_y_above_lin_reg_s, df2_copy.index[2:])
    pd_buy_x_below_lin_reg_m = pd.Series(buy_x_below_lin_reg_m, df1_copy.index[2:])
    pd_buy_y_above_lin_reg_m = pd.Series(buy_y_above_lin_reg_m, df2_copy.index[2:])


    dfzs = ( ( res_data - res_data.mean() ) / res_data.std() )
    dfzs_s = ( ( res_data_s - res_data_s.mean() ) / res_data_s.std() )
    dfzs_m = ( ( res_data_m - res_data_m.mean() ) / res_data_m.std() )

    # opposite_date_index = []
    # for idx in range(6, (len(x)-6) + 1):  
    #     idx2 = idx + start
    #     if check_opposite_signs_fixed(x, y, idx2):
    #         opposite_date_index.append(idx2)
    # print(opposite_date_index)
    # for idx in range(6, (len(x)-6) + 1):  # Start from 3 to ensure we have at least 2 previous days.
    #     idx2 = idx + start
    #     # print(idx2)
    #     # print(x[idx2])
    #     if check_opposite_signs(x, y, idx2):
    #         # opposite_dates.append(df1.iloc[idx-1]['Date'])
    #         opposite_date_index.append(idx2)
    #         print(idx2)

    # Filter dataframes for opposite_dates
    # df1_opposite = x[df1['Date'].isin(opposite_dates)]
    # df2_opposite = y[df2['Date'].isin(opposite_dates)]
    # print(len(df1_opposite))

    # df1_normalized_filtered, df2_normalized_filtered 
    # df1_n_minmax_filtered,   df2_n_minmax_filtered
    # df1_filtered,            df2_filtered

    # Main plot
    axs[0].plot(df1_copy['Close'], label='Intraday F')
    axs[0].plot(df2_copy['Close'], label='Intraday GM')
    axs[0].legend()

    axs[1].plot(df1_start['Close'], label='F normalized start')
    axs[1].plot(df2_start['Close'], label='GM normalized start')
    axs[1].legend()

    axs[2].plot(df1_n_minmax['Close'], label='F minmax')
    axs[2].plot(df2_n_minmax['Close'], label='GM minmax')
    axs[2].legend()

    # axs[3].plot(res_data, label='res_data')
    # axs[3].plot(res_ema, label='res_ema')
    # axs[3].axhline(0, color='red') # Add horizontal line at y=0 for reference 
    # axs[3].axhline(1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[3].axhline(-1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[3].axhline(2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[3].axhline(-2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[3].legend()

    # axs[4].plot(res_data_s, label='res_data_s')
    # axs[4].plot(res_ema_s, label='res_ema_s')
    # axs[4].axhline(0, color='red') # Add horizontal line at y=0 for reference 
    # axs[4].axhline(1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[4].axhline(-1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[4].axhline(2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[4].axhline(-2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[4].legend()

    # axs[5].plot(res_data_m, label='res_data_m')
    # axs[5].plot(res_ema_m, label='res_ema_m')
    # axs[5].axhline(0, color='red') # Add horizontal line at y=0 for reference 
    # axs[5].legend()

    # axs[6].plot(zscore, label='zscore')
    # axs[6].axhline(0, color='red') # Add horizontal line at y=0 for reference
    # axs[6].axhline(1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[6].axhline(-1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[6].axhline(2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[6].axhline(-2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[6].legend()

    # axs[7].plot(zscore_s, label='zscore_s ')
    # axs[7].axhline(0, color='red') # Add horizontal line at y=0 for reference
    # axs[7].axhline(1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[7].axhline(-1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[7].axhline(2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[7].axhline(-2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[7].legend()

    # axs[8].plot(zscore_m, label='zscore_m')
    # axs[8].axhline(0, color='red') # Add horizontal line at y=0 for reference
    # axs[8].legend()

    # axs[9].plot(dfzs, label='dfzs')
    # axs[9].axhline(0, color='red') # Add horizontal line at y=0 for reference
    # axs[9].axhline(1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[9].axhline(-1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[9].axhline(2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[9].axhline(-2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[9].legend()

    # axs[10].plot(dfzs_s, label='dfzs_s')
    # axs[10].axhline(0, color='red') # Add horizontal line at y=0 for reference
    # axs[10].axhline(1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[10].axhline(-1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[10].axhline(2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[10].axhline(-2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[10].legend()

    # axs[11].plot(dfzs_m, label='dfzs_m')
    # axs[11].axhline(0, color='red') # Add horizontal line at y=0 for reference
    # axs[11].axhline(1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[11].axhline(-1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[11].axhline(2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[11].axhline(-2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[11].legend()

    # axs[9].plot(res_ema, label='res_ema')
    # axs[9].axhline(0, color='red') # Add horizontal line at y=0 for reference
    # axs[9].axhline(1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[9].axhline(-1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[9].axhline(2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[9].axhline(-2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[9].legend()

    # axs[10].plot(res_ema_s, label='res_ema_s')
    # axs[10].axhline(0, color='red') # Add horizontal line at y=0 for reference
    # axs[10].axhline(1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[10].axhline(-1, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[10].axhline(2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[10].axhline(-2, linestyle='--', color='gray') # Add horizontal line at y=0 for reference 
    # axs[10].legend()

    # axs[11].plot(res_ema_m, label='res_ema_m')
    # axs[11].axhline(0, color='red') # Add horizontal line at y=0 for reference
    # axs[11].legend()



    # axs[5].plot(df1_copy['Close'], color='red', label='Intraday F')
    # axs[5].plot(df2_copy['Close'], color='red', label='Intraday GM')
    # axs[5].axhline(0, color='green') # Add horizontal line at y=0 for reference
    # axs[5].plot(combo_dates_x, color='green')
    # axs[5].plot(combo_dates_y, color='green')
    # # axs[0].scatter(x_points, y_points, color='red', marker='o', label='Points')
    # axs[3].legend()

    # axs[6].axhline(0, color='red') # Add horizontal line at y=0 for reference
    # axs[6].plot(df1_copy['Close'], color='black', label='Intraday F')
    # axs[6].plot(combo_dates_x, color='green', label='Intraday GM')
    # axs[6].plot(pd_date_lin_reg_x, label='F (lin_reg_x)')
    # axs[6].legend()

    # axs[7].axhline(0, color='red') # Add horizontal line at y=0 for reference
    # axs[7].plot(df2_copy['Close'], color='black', label='Intraday GM')
    # axs[7].plot(combo_dates_y, color='red', label='Intraday GM')
    # axs[7].plot(pd_date_lin_reg_y, label='GM (lin_reg_y)')
    # axs[7].legend()

    # # pd_buy_x_below_lin_reg = pd.Series(buy_x_below_lin_reg, df1.index[2:])
    # # pd_buy_y_above_lin_reg = pd.Series(buy_y_above_lin_reg, df2.index[2:])
    # axs[8].scatter(pd_buy_x_below_lin_reg.index, pd_buy_x_below_lin_reg.values, label='res < 0 BUY F SELL GM')
    # axs[8].axhline(0, color='red') # Add horizontal line at y=0 for reference
    # axs[8].legend()

    # # print("len(pd_buy_y_above_lin_reg.keys)", len(pd_buy_y_above_lin_reg.keys))
    # axs[9].scatter(pd_buy_y_above_lin_reg.index,pd_buy_y_above_lin_reg.values, label='res > 0 SELL F BUY GM')
    # axs[9].axhline(0, color='red') # Add horizontal line at y=0 for reference
    # axs[9].legend()

    plt.show()

def main():

    # stock_symbol_1 = 'F'
    # stock_symbol_2 = 'GM'
    # start_date = '2013-01-01'
    # end_date = '2014-06-01'
    # stock_data_1 = yf.download(ticker=stock_symbol_1, start=start_date, end=end_date)
    # stock_data_2 = yf.download(ticker=stock_symbol_2, start=start_date, end=end_date)
    # df1 = read_and_normalize(stock_data_1, start_date)
    # df2 = read_and_normalize(stock_data_2, end_date)

    start = 1000   # change these to the values you want
    end = 2000   # change these to the values you want

    # Read and normalize data at start
    df1_normalized, df1_n_minmax, df1 = read_and_normalize('/Users/walkerhutchinson/Desktop/ThunderFund/meteor_code/F_2014_15min.csv', start)
    df2_normalized, df2_n_minmax, df2 = read_and_normalize('/Users/walkerhutchinson/Desktop/ThunderFund/meteor_code/GM_2014_15min.csv', start)
    
    # Filter by index
    df1_start, df1_n_minmax, df1_copy = filter_by_index(df1_normalized, df1_n_minmax, df1, start, end)
    df2_start, df2_n_minmax, df2_copy = filter_by_index(df2_normalized, df2_n_minmax, df2, start, end)

    # Plot
    plot(df1_start, df1_n_minmax, df1_copy, df2_start, df2_n_minmax, df2_copy, start)

if __name__ == "__main__":
    main()


SyntaxError: invalid syntax (1279104706.py, line 421)

In [48]:
from ibapi.wrapper import EWrapper
from ibapi.client import EClient
from ibapi.contract import Contract
from ibapi.order import Order

import numpy as np

ticker_groups = [ ['AAPL','SPY'], ["GME","AMC"], ["F","GM"] ]    

def create_contract(symbol):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'STK' 
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    return contract

class RollingWindow:
    def __init__(self, tickers, n):
        self.n = n
        self.windows = {ticker: np.full(n, np.nan) for ticker in tickers}

    def update(self, ticker, price):
        if ticker in self.windows:
            self.windows[ticker] = np.roll(self.windows[ticker], shift=-1)
            self.windows[ticker][-1] = price

    def get_window(self, ticker):
        return self.windows.get(ticker)

contracts_groups = [[create_contract(ticker) for ticker in group] for group in ticker_groups]
all_tickers = [ticker for group in ticker_groups for ticker in group]
rw = RollingWindow(all_tickers, 10)

print(vars(rw).keys())
print()
print(vars(rw).values())
print()

rw.update("AAPL", 100)  
print(rw.get_window(ticker='AAPL'))


dict_keys(['n', 'windows'])

dict_values([10, {'AAPL': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'SPY': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'GME': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'AMC': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'F': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]), 'GM': array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])}])

[ nan  nan  nan  nan  nan  nan  nan  nan  nan 100.]


In [51]:
from ibapi.wrapper import EWrapper
from ibapi.client import EClient
from ibapi.contract import Contract
from ibapi.order import Order

import pandas as pd
import numpy as np

ticker_groups = [ ['AAPL','SPY'], ["GME","AMC"], ["F","GM"] ]    

def create_contract(symbol):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'STK' 
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    return contract

class RollingWindow:
    def __init__(self, tickers, n):
        self.n = n
        self.windows = pd.DataFrame({ticker: pd.Series([np.nan] * n) for ticker in tickers})
        # self.windows = pd.DataFrame({ticker: pd.Series([pd.NA] * n) for ticker in tickers})

    def update(self, ticker, price):
        if ticker in self.windows:
            self.windows[ticker] = self.windows[ticker].shift(-1)
            self.windows.at[self.n - 1, ticker] = price

    def get_window(self, ticker):
        return self.windows.get(ticker)

contracts_groups = [[create_contract(ticker) for ticker in group] for group in ticker_groups]
all_tickers = [ticker for group in ticker_groups for ticker in group]
rw = RollingWindow(all_tickers, 10)

print(vars(rw).keys())
print()
print(vars(rw).values())
print()

rw.update("SPY", 100)  
print(rw.get_window(ticker='SPY'))
print(rw.get_window(ticker='AAPL'))


dict_keys(['n', 'windows'])

dict_values([10,    AAPL  SPY  GME  AMC   F  GM
0   NaN  NaN  NaN  NaN NaN NaN
1   NaN  NaN  NaN  NaN NaN NaN
2   NaN  NaN  NaN  NaN NaN NaN
3   NaN  NaN  NaN  NaN NaN NaN
4   NaN  NaN  NaN  NaN NaN NaN
5   NaN  NaN  NaN  NaN NaN NaN
6   NaN  NaN  NaN  NaN NaN NaN
7   NaN  NaN  NaN  NaN NaN NaN
8   NaN  NaN  NaN  NaN NaN NaN
9   NaN  NaN  NaN  NaN NaN NaN])

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9    100.0
Name: SPY, dtype: float64
0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
8   NaN
9   NaN
Name: AAPL, dtype: float64
